In [1]:
!pip install psycopg2

     ---------------------------------------- 1.2/1.2 MB 7.4 MB/s eta 0:00:00


In [2]:
import psycopg2
from collections import defaultdict

#plug these variables into pgadmin to manage the DB from there
host = 'database-1.crkok30yxyiy.ap-southeast-1.rds.amazonaws.com'
sample_database = 'postgres'
user='postgres'
password = 'Password1'

In [3]:
class Utility():
    def __init__(self):
        self.datoms = []
        self.entity_id = 0
    
    def rstrip_col(self, c):
        return c.rstrip() if isinstance(c, str) else str(c).rstrip()

    def rstrip_row(self, r):
        return list(map(lambda c: self.rstrip_col(c), r))

    def execute(self, query):
        print(query)
        cursor.execute(query)
        sql_res = cursor.fetchall()
        return list(map(lambda r: self.rstrip_row(r), sql_res))

    def print_rows(self, sql_res):
        for row in sql_res: 
            print(row)
        print()

    def print(self, query, table_name):
        table_schema = self.execute("SELECT column_name, data_type, is_nullable FROM information_schema.columns WHERE table_name = '%s' ORDER BY ordinal_position;" % (table_name))
        self.print_rows(table_schema)
        sql_res = self.execute(query)
        self.print_rows(sql_res)
        print()

    def get_table_headers(self, table_name): 
        return list(map(lambda c: c[0], self.execute("SELECT column_name FROM information_schema.columns WHERE table_name = '%s' ORDER BY ordinal_position;" % (table_name))))

    def convert_to_datoms(self, sql_res, table_name):
        table_headers = self.get_table_headers(table_name)
        datoms = []
        for row in sql_res:   
            # row = ['A007', 'Ramasundar', 'Bangalore', '0.15', '077-25814763', '']
            for i in range(len(row)):
                entity_id = str(self.entity_id)
                attribute = ":%s/%s" % (table_name, table_headers[i])
                value = row[i]
                transaction_id = ""
                datom = (entity_id, attribute, value, transaction_id)
                datoms.append(datom)
            self.entity_id += 1
        return datoms

    def translate_to_sql_query(self, attribute):  # KIV enhancement to specify table_attribute
        # attribute = ":AGENTS/AGENT_NAME"
        table_name, table_attribute = attribute[1:].split('/')
        # return "Select \"%s\" from \"%s\"" % (table_attribute, table_name) 
        return "Select %s from \"%s\"" % ("*", table_name)  

    def append(self, table_name):
        sql_res0 = self.execute('Select * from "%s"' % (table_name))
        for datom in self.convert_to_datoms(sql_res0, table_name):
            self.datoms.append(datom)


In [4]:
class DataPatternResolver():
# fn prefixed with "_" = private fn
    def __init__(self):
        self.util = Utility()

    def _prepend_entity_id(self, data_pattern):
        if data_pattern[0][0] != ":": return
        data_pattern.insert(0, "_")

    def _reappend_value(self, data_pattern):
        L = len(data_pattern)
        if L < 4: return 
        stack = []
        for i in range(2, L):
            stack.append(data_pattern.pop())
        reversed_stack = []
        for i in range(len(stack)):
            reversed_stack.append(stack.pop())
        data_pattern.append(" ".join(reversed_stack))

    def _parse_data_pattern(self, d):
        # d = "?c :AGENTS/AGENT_NAME "Alex""
        data_pattern0 = d.split()
        data_pattern1 = list(map(lambda val: val.strip("\""), data_pattern0))
        self._prepend_entity_id(data_pattern1)
        self._reappend_value(data_pattern1)
        return data_pattern1

    def _is_attribute(self, val):
        prefix = None if len(val) == 0 else val[0]
        return prefix == ":"

    def _is_constant(self, val):
        prefix = None if len(val) == 0 else val[0]
        return prefix and prefix != "?" and prefix != ":"

    def _is_variable(self, val):
        prefix = None if len(val) == 0 else val[0]
        return prefix == "?"

    def _is_bounded_variable(self, val, variables):
        return self._is_variable(val) and val[1:] in variables

    def _is_matching_attribute(self, pos, datom, data_pattern):
        attr_dt = datom[pos]
        attr_dp = data_pattern[pos]
        return attr_dt == attr_dp or attr_dt[:len(attr_dp)] == attr_dp

    def _is_matching_constant(self, pos, datom, data_pattern):
        const_dt = datom[pos]
        const_dp = data_pattern[pos]
        return const_dt == const_dp or const_dp == "_"

    def _is_matching_variable(self, pos, datom, data_pattern, variables):
        key = data_pattern[pos][1:]
        return datom[pos] in variables[key]

    def _check(self, flags, data_pattern, fn, *args):
        for i in range(len(data_pattern)):
            val = data_pattern[i]
            if not fn(val, *args): continue
            flags[i] = True

    def _filter(self, flags, datoms, data_pattern, fn, *args):
        res = []
        for datom in datoms:
            match = True
            for i in range(4):
                if not flags[i]: continue
                if not fn(i, datom, data_pattern, *args): match = False
            if not match: continue
            res.append(datom)
        return res

    def _match_attribute(self, datoms, data_pattern):
        flags = [False] * 4
        self._check(flags, data_pattern, self._is_attribute)
        print("ATTRIBUTE:", flags)
        if not any(flags): return datoms
        return self._filter(flags, datoms, data_pattern, self._is_matching_attribute)

    def _match_constants(self, datoms, data_pattern):
        flags = [False] * 4
        self._check(flags, data_pattern, self._is_constant)
        print("CONSTANTS:", flags)
        if not any(flags): return datoms
        return self._filter(flags, datoms, data_pattern, self._is_matching_constant)

    def _match_variables(self, datoms, data_pattern, variables):
        if not variables.keys(): return datoms
        flags = [False] * 4
        self._check(flags, data_pattern, self._is_bounded_variable, variables)
        print("BOUNDED VARIABLES:", flags)
        if not any(flags): return datoms
        return self._filter(flags, datoms, data_pattern, self._is_matching_variable, variables)
        
    def _bind_variables(self, datoms, data_pattern, variables, visited):
        flags = [False] * 4
        self._check(flags, data_pattern, self._is_variable)
        print("TO-BE-BOUNDED VARIABLES:", flags)
        if not any(flags): return 
        for datom in datoms:
            for i in range(4):
                if not flags[i]: continue
                key = data_pattern[i][1:]
                val = datom[i]
                if val in visited[key]: continue
                variables[key].append(val)
                visited[key].add(val)
    
    def _resolve(self, datoms, data_pattern, variables, visited):
        # data_pattern = ['?c', ':AGENTS/AGENT_NAME', 'Alex', '']
	    # variables = {c: {'A007', 'A003', 'A008'}}
        filtered_datoms0 = self._match_attribute(datoms, data_pattern)
        # self.util.print_rows(filtered_datoms0)
        filtered_datoms1 = self._match_constants(filtered_datoms0, data_pattern)
        filtered_datoms2 = self._match_variables(filtered_datoms1, data_pattern, variables)
        self._bind_variables(filtered_datoms2, data_pattern, variables, visited)
        print(variables)
    
    def resolve(self, datoms, data_patterns):
	    # data_patterns = ['?c :AGENTS/WORKING_AREA ?w', '?c :AGENTS/WORKING_AREA ?w', '?c :AGENTS/WORKING_AREA ?w']
        variables = defaultdict(list)
        visited = defaultdict(set)
        for i in range(len(data_patterns)):
            d = data_patterns[i]
            data_pattern = self._parse_data_pattern(d)
            print("DATA PATTERN %d: " % (i), data_pattern)
            self._resolve(datoms, data_pattern, variables, visited)
            print()
        return variables

In [9]:
conn = psycopg2.connect(host = host, database = sample_database, user = user, password = password)
cursor = conn.cursor()
u = Utility()
query = u.translate_to_sql_query(":CUSTOMER/CUST_NAME") + " WHERE \"CUST_NAME\" IN ('Ravindran', 'Srinivas', 'Rangarappa', 'Venkatpati')"
u.print_rows(u.execute(query))

Select * from "CUSTOMER" WHERE "CUST_NAME" IN ('Ravindran', 'Srinivas', 'Rangarappa', 'Venkatpati')
['C00025', 'Ravindran', 'Bangalore', 'Bangalore', 'India', '2', '5000.00', '7000.00', '4000.00', '8000.00', 'AVAVAVA', 'A011']
['C00017', 'Srinivas', 'Bangalore', 'Bangalore', 'India', '2', '8000.00', '4000.00', '3000.00', '9000.00', 'AAAAAAB', 'A007']
['C00014', 'Rangarappa', 'Bangalore', 'Bangalore', 'India', '2', '8000.00', '11000.00', '7000.00', '12000.00', 'AAAATGF', 'A001']
['C00016', 'Venkatpati', 'Bangalore', 'Bangalore', 'India', '2', '8000.00', '11000.00', '7000.00', '12000.00', 'JRTVFDD', 'A007']



In [10]:
# Sample Postgres Connection
conn = psycopg2.connect(host = host, database = sample_database, user = user, password = password)
cursor = conn.cursor()

dpr = DataPatternResolver()
u = dpr.util
# u.print("SELECT %s FROM \"%s\"" % ("*","AGENTS"), "AGENTS")  # note format: "\"AGENT_CODE\""
# u.print("SELECT %s FROM \"%s\"" % ("*","CUSTOMER"), "CUSTOMER")  # note format: "\"CUST_CODE\""
# u.print("SELECT %s FROM \"%s\"" % ("*","ORDERS"), "ORDERS")  # note format: "\"ORD_NUM\""
u.append("AGENTS")
u.append("CUSTOMER")
u.append("ORDERS")
print("len of datoms: ", len(u.datoms))
print()
# u.print_rows(u.datoms)

Select * from "AGENTS"
SELECT column_name FROM information_schema.columns WHERE table_name = 'AGENTS' ORDER BY ordinal_position;
Select * from "CUSTOMER"
SELECT column_name FROM information_schema.columns WHERE table_name = 'CUSTOMER' ORDER BY ordinal_position;
Select * from "ORDERS"
SELECT column_name FROM information_schema.columns WHERE table_name = 'ORDERS' ORDER BY ordinal_position;
len of datoms:  610



In [11]:
data_patterns0 = ['?a_id :AGENTS/WORKING_AREA "Bangalore"', '?a_id :AGENTS/AGENT_CODE ?a_code','?c_id :CUSTOMER/AGENT_CODE ?a_code', '?c_id :CUSTOMER/CUST_NAME ?c_name']
data_patterns1 = ['?a_id :AGENTS/WORKING_AREA "New York"', '?a_id :AGENTS/AGENT_CODE ?a_code', '?c_id :CUSTOMER/AGENT_CODE ?a_code', '?c_id :CUSTOMER/CUST_NAME ?c_name']
data_patterns2 = ['?c_id :CUSTOMER/CUST_CITY "San Jose"', '?c_id :CUSTOMER/CUST_CITY "San Jose"', '?o_id :ORDERS/CUST_CODE ?c_id']
data_patterns3 = ['?c_id :CUSTOMER/CUST_CITY "New York"']
data_patterns4 = ['"0" ?attr ?val']
data_patterns5 = [':AGENTS/AGENT_NAME ?name']
data_patterns6 = ['?A :AGENTS/AGENT_NAME "Ivan"', '?A :AGENTS/AGENT_CODE ?B', '?C :ORDERS/AGENT_CODE ?B', '?C :ORDERS/CUST_CODE ?D', '?E :CUSTOMER/CUST_CODE ?D']
# SELECT "AGENT_NAME" FROM "AGENTS" WHERE 
dpr.resolve(dpr.util.datoms, data_patterns0)
print()

DATA PATTERN 0:  ['?a_id', ':AGENTS/WORKING_AREA', 'Bangalore']
ATTRIBUTE: [False, True, False, False]
CONSTANTS: [False, False, True, False]
TO-BE-BOUNDED VARIABLES: [True, False, False, False]
defaultdict(<class 'list'>, {'a_id': ['0', '3', '7']})

DATA PATTERN 1:  ['?a_id', ':AGENTS/AGENT_CODE', '?a_code']
ATTRIBUTE: [False, True, False, False]
CONSTANTS: [False, False, False, False]
BOUNDED VARIABLES: [True, False, False, False]
TO-BE-BOUNDED VARIABLES: [True, False, True, False]
defaultdict(<class 'list'>, {'a_id': ['0', '3', '7'], 'a_code': ['A007', 'A011', 'A001']})

DATA PATTERN 2:  ['?c_id', ':CUSTOMER/AGENT_CODE', '?a_code']
ATTRIBUTE: [False, True, False, False]
CONSTANTS: [False, False, False, False]
BOUNDED VARIABLES: [False, False, True, False]
TO-BE-BOUNDED VARIABLES: [True, False, True, False]
defaultdict(<class 'list'>, {'a_id': ['0', '3', '7'], 'a_code': ['A007', 'A011', 'A001'], 'c_id': ['14', '28', '33', '34']})

DATA PATTERN 3:  ['?c_id', ':CUSTOMER/CUST_NAME', '?c

In [12]:
import re

txt0 = ":find ?name :where [?m :movie/title 'Lethal Weapon'] [?m :movie/cast ?p] [?p :person/name ?name]"
txt1 = ":find ?title :where [_ :movie/title ?title]"
matcher = re.compile("(?<=\[)(.*?)(?=\])")
res = matcher.findall(txt0)
for _match in res:
    print(_match)

?m :movie/title 'Lethal Weapon'
?m :movie/cast ?p
?p :person/name ?name
